In [1]:
# !pip install nltk==3.5
# !pip install colorama==0.4.3
# !pip install numpy==1.18.5
# !pip install scikit_learn==0.23.2
# !pip install Flask==1.1.2
#https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

In [3]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import plot_model


In [5]:


# Check the GPU available
print(tf.config.list_physical_devices())
print("is_built_with_cuda:",tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
is_built_with_cuda: True


In [7]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Loading Json file 

In [8]:
with open('intent.json') as file:
 data=json.load(file)

In [ ]:
training_sentences=[]
training_labels=[]
labels=[]
responses=[]
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])     
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
        
num_classes=len(labels)        
    
    
training_sentences   
responses
labels

In [ ]:
lbl_encoder=LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels=lbl_encoder.transform(training_labels)
print(training_labels)

In [ ]:

vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model = Sequential()
#https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/#:~:text=The%20output%20of%20the%20Embedding,vector%20using%20the%20Flatten%20layer.
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
    
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='sequential_model.png', show_shapes=True, show_dtype=False,
        show_layer_names=True, rankdir='TB', expand_nested=True, dpi=80 )

In [ ]:

# epochs = 500
# history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)
#  train our model
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels),  validation_split=0.25, epochs=epochs, batch_size=16, verbose=1)

In [ ]:
# Explore History details
print(history.params)
print(history.history.keys())
print('Final training loss \t', history.history['loss'][-1])
print('Final training accuracy ',history.history['accuracy'][-1])

### Pickle can be used to serialize Python object structures, which refers to the process of converting an object in the memory to a byte stream that can be stored as a binary file on disk. When we load it back to a Python program, this binary file can be de-serialized back to a Python object.

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as pyplot
# summarize history for accuracy
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='upper left')
pyplot.savefig('intent_accuracy.png')
pyplot.show()


# summarize history for loss
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='upper left')
pyplot.savefig('intent_loss.png')
pyplot.show()

# plot mse during training
pyplot.subplot(212)
pyplot.title('SparseC Categorical Crossentropy')
pyplot.plot(history.history['sparse_categorical_crossentropy'], label='train')
# pyplot.plot(history.history['val_sparse_categorical_crossentropy'], label='test')
pyplot.legend()
pyplot.savefig('intent_mse.png')
pyplot.show()

In [ ]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intent.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
        print (lbl_encoder)
    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        #print(CLASSES[np.argmax(result)])
        print([np.argmax(result)])
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        print("****************")
        print(tag)
#         print(data['intents'])
        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

In [ ]:
hello

